In [1]:
# %%
import time
import numpy as np
import pickle
from numpy.linalg import det

import CMINE_lib as CMINE
# from Guassian_variables import Data_guassian

import pandas as pd
from scipy.stats import multivariate_normal
import itertools

np.random.seed(37)
from scipy import stats
from sklearn.neighbors import KernelDensity

import math

import torch 
import torch.nn as nn
import torch.nn.functional as F

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


In [2]:
def log_sum_exp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation
    value.exp().sum(dim, keepdim).log()
    """
    # TODO: torch.max(value, dim=None) threw an error at time of writing
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        if isinstance(sum_exp, Number):
            return m + math.log(sum_exp)
        else:
            return m + torch.log(sum_exp)

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
from modules.CMI import DR_CMI, CDL_CMI

In [5]:
Dim = 5
batch_size = 64
#dataset = CMINE.create_dataset_DGP( Dim=5, N=batch_size)
dataset = CMINE.create_dataset_DGP_binary_A( Dim=5, N=batch_size)
s_t = torch.from_numpy(dataset[0]).float().cuda()
s_next = torch.from_numpy(dataset[1]).float().cuda()
a = torch.from_numpy(dataset[2]).float().cuda()

In [6]:
sample_dim = 2*Dim

hidden_size = 15
learning_rate = 0.005
training_steps = 10

cubic = False 


In [7]:
def train_esm(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_dr = DR_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_dr = torch.optim.Adam(model_dr.parameters(), learning_rate)
    dr_est_values = []
    
    model_cdl = CDL_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_cdl = torch.optim.Adam(model_cdl.parameters(), learning_rate)
    cdl_est_values = []
    
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = CMINE.create_dataset_DGP_binary_A(Dim=Dim, N=N)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_dr.eval()
        drs = model_dr(batch_x, batch_y)
        #mi_est_values.append(cmi)
        dr_est_values.append(drs)
        model_dr.train() 

        model_loss = model_dr.learning_loss(batch_x, batch_y)

        optimizer_dr.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_dr.step()
        
        model_cdl.eval()
        cdl_cmi = model_cdl(batch_x, batch_y)
        cdl_est_values.append(cdl_cmi)
        model_cdl.train() 

        model_loss = model_cdl.learning_loss(batch_x, batch_y)

        optimizer_cdl.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_cdl.step()


        del batch_x, batch_y
        torch.cuda.empty_cache()
    return dr_est_values, cdl_est_values

In [9]:
# def train_cdl(N = 64, training_steps = 10):
#     torch.cuda.empty_cache()
#     model_cdl = CDL_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
#     optimizer_cdl = torch.optim.Adam(model_cdl.parameters(), learning_rate)
#     cdl_est_values = []
#     for step in range(training_steps):
#         #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
#         dataset = CMINE.create_dataset_DGP_binary_A(Dim=Dim, N=N)
#         s_t = torch.from_numpy(dataset[0]).float().cuda()
#         s_next = torch.from_numpy(dataset[1]).float().cuda()
#         a = torch.from_numpy(dataset[2]).float().cuda()

#         batch_x = torch.cat([s_t,a], dim=1)
#         batch_y = s_next
#         model_cdl.eval()
#         cdl_cmi = model_cdl(batch_x, batch_y)
#         cdl_est_values.append(cdl_cmi)
#         model_cdl.train() 

#         model_loss = model_cdl.learning_loss(batch_x, batch_y)

#         optimizer_cdl.zero_grad()
#         model_loss.backward(retain_graph=True)
#         optimizer_cdl.step()

#         del batch_x, batch_y
#         torch.cuda.empty_cache()
#     return cdl_est_values

In [8]:
dr_est_values, cdl_est_values = train_esm()


In [9]:
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.03772729e-01 1.01849623e+00 7.97322467e-01 2.54387358e-01
 2.68622077e-01 3.48189807e+00 4.64998634e+02 2.83578072e+01
 1.86894889e+04 3.29520065e+02]
[7.57488118e+01 4.65357479e+01 6.01218122e+01 2.06809561e+01
 7.07125411e+01 3.76959266e+03 5.86597748e+04 1.50011069e+05
 1.54557170e+05 6.52319499e+06]


In [11]:
N = 128
dr_est_values, cdl_est_values = train_esm(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[3.66495641 0.48939352 0.10916508 0.5238816  0.79923622 0.48864229
 0.35026615 1.66584601 1.44448828 5.24435752]
[ 4.78474628  0.62300313  0.24407864  0.70721587  1.00285186  1.18958928
  1.82694086  4.67280312 16.82175357  7.98338467]


In [12]:
N = 32
dr_est_values, cdl_est_values = train_esm(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[0.15128056 0.34122159 0.14039005 0.14874211 0.14583907 2.93205448
 7.25294628 0.29748212 4.14152377 0.53155779]
[ 0.41654251  0.74857839  0.31920083  0.35007294  0.31365633  5.96694219
 16.56601878  0.41881651  8.6066275   1.0919261 ]


In [13]:
N = 16
dr_est_values, cdl_est_values = train_esm(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[0.24692184 2.63111432 0.23171107 5.89867871 0.42849664 0.73036352
 0.51368312 0.80600969 0.90605433 0.67817194]
[ 0.50548669  5.78074821  0.42121193 14.04263662  0.85493     1.39348036
  1.42014043  1.06013892  1.59416723  1.72782246]


# Training Step

In [14]:

training_step = 100
N = 16
dr_est_values, cdl_est_values = train_esm(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[3.03880225e+06 1.57874214e+02 1.48158180e+01 1.19313124e+02
 8.01108421e+00 1.95269468e+08 2.30917846e+06 4.95195440e+04
 1.63975055e+05 6.93404654e+02]
[1.54906631e+06 1.39800087e+04 2.20698382e+04 2.61903255e+04
 2.05111227e+04 7.07911130e+07 5.30454029e+10 7.00310593e+06
 2.06086847e+08 1.98817036e+07]


In [15]:
training_step = 100
N = 32
dr_est_values, cdl_est_values = train_esm(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.17077079e+03 1.59495717e+02 7.37845760e+01 5.59349477e+02
 3.27837828e+02 2.30278616e+07 1.42276261e+04 6.17422289e+03
 2.92034102e+05 9.02656667e+04]
[2.90436030e+04 1.20230923e+04 1.06401160e+04 1.43187511e+04
 1.20261286e+04 3.03978297e+11 3.99767017e+07 1.17134363e+07
 9.21116356e+10 8.38699038e+08]


In [16]:
training_step = 100
N = 64
dr_est_values, cdl_est_values = train_esm(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[4.07220035e+03 3.54165966e+06 6.69968124e+03 8.08036271e+03
 4.21918407e+03 3.73227067e+05 5.58577197e+03 1.62420501e+04
 9.19441038e+02 2.69391539e+07]
[2.93097251e+06 2.48700379e+08 1.68484774e+05 1.80279023e+07
 3.97901035e+07 1.47212191e+07 7.70243576e+06 1.40382615e+07
 7.14141578e+05 3.27662717e+10]


In [17]:
training_step = 100
N = 128
dr_est_values, cdl_est_values = train_esm(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[5.16158611e+04 7.02213563e+04 2.42981604e+06 4.98532512e+04
 5.51557428e+04 5.39883304e+08 6.39579711e+12 5.90482473e+07
 1.79597423e+12 1.30359451e+05]
[3.99136984e+08 3.93590577e+08 1.10049691e+10 3.93357801e+08
 3.93447897e+08 2.42371052e+12 3.40231801e+12 1.42715037e+10
 1.25344576e+14 1.40457691e+09]


In [18]:
N = 526
training_step = 100
dr_est_values, cdl_est_values = train_esm(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.79390223e+03 2.02561030e+03 2.00996960e+03 2.01279224e+03
 8.29084961e+03 9.82168962e+04 5.41656019e+04 5.35609346e+03
 6.44758298e+07 1.70287555e+08]
[1.41368943e+05 7.01262309e+04 7.04075745e+04 7.00891153e+04
 1.35851496e+05 1.73736456e+08 4.69747733e+08 2.26369548e+07
 3.04493641e+09 6.72284618e+12]
